In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


# Deep Neural Networks

## Session 08b - Lecture

## Neural Network with ${Swish}$ activation function
- Single hidden layer
- **${Swish}$** activation function
- Multi-class output
- FIFA dataset 

<img src='../../images/prasami_color_tutorials_small.png' width='400' alt="By Pramod Sharma : pramod.sharma@prasami.com" align="left"/>

In [ ]:
# Lets import some libraries
import os

import numpy as np

import pandas as pd

from time import time

import matplotlib.pyplot as plt

import pickle


from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

%matplotlib inline

In [ ]:
# Some basic parameters
inpDir = '../input'
outDir = '../output'
modelDir = '../models'
subDir = 'fifa_2019'

RANDOM_STATE = 24

np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results

EPOCHS = 40001 # number of epochs
ALPHA = 0.1 # learning rate
TEST_SIZE = 0.2

SAVE_MODEL = True # for saving the model

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = plt.cm.Dark2

plt.rcParams.update(params)

## Read FIFA 2019 data set

In [ ]:
data_df = pd.read_csv(os.path.join(inpDir, 'fifa_2019.csv'))
data_df.shape

In [ ]:
data_df.head()

In [ ]:
# cols = data_df.columns

# for col in cols:
#    print('\nColumn Name ', col, ':', data_df[col].unique())

In [ ]:
# removing rows with position = null
data_df = data_df[data_df["Position"].notnull()]
data_df.head()

In [ ]:
# Following columns appear to be relevant for our analysis
rel_cols = ["Position", 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
            'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
            'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
            'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
            'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
            'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
            'GKKicking', 'GKPositioning', 'GKReflexes']

In [ ]:
data_df = data_df[rel_cols]
data_df.head()

In [ ]:
# Are there NaNs Still?

data_df.isna().sum()

In [ ]:
goalkeeper = 'GK'
forward = ['ST', 'LW', 'RW', 'LF', 'RF', 'RS','LS', 'CF']
midfielder = ['CM','RCM','LCM', 'CDM','RDM','LDM', 'CAM', 'LAM', 'RAM', 'RM', 'LM']
defender = ['CB', 'RCB', 'LCB', 'LWB', 'RWB', 'LB', 'RB']

In [ ]:
#Assign labels to goalkeepers
data_df.loc[data_df["Position"] == "GK", "Position"] = 0

#Defenders
data_df.loc[data_df["Position"].isin(defender), "Position"] = 1

#Midfielders
data_df.loc[data_df["Position"].isin(midfielder), "Position"] = 2

#Forward
data_df.loc[data_df["Position"].isin(forward), "Position"] = 3

In [ ]:
# Convert Column "Position" to numeric so that Pandas does not complain
data_df['Position'] = pd.to_numeric(data_df['Position'], downcast="integer")

In [ ]:
labels = {0: 'Goal Keeper', 1: 'Defender', 2: 'Mid-Fielder', 3: 'Forward'}

In [ ]:
y = data_df["Position"].values

X = data_df.drop("Position", axis = 1)

In [ ]:
# one hot encoding
y = pd.get_dummies(y).values
y

In [ ]:
#  Split the data in training and test sets to measure performance of the model.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, 
                                                    # shuffle=True, # This is True by Default
                                                    stratify=y,
                                                    random_state=RANDOM_STATE )

print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

# Prepare model

## Activation and Softmax

Softmax code is for demo purpose only. In general softmax functions can be highly unstable.
If you must implement softmax, use libraries such as scipy.special.softmax. 

In [ ]:
def fn_softmax(z):
    
    '''
    Args:
        z : a matrix of z values of shape (m, n_output)
    returns:
        Softmax values of z
    
    '''
    
    exp_scores = np.exp(z)

    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    

In [ ]:
def fn_actv(z):
    '''
    Args:
        z: array z value
    Returns:
        a: array - activations
    '''
    return np.tanh(z)

def fn_actv_prime(z):
    '''
    Args:
        z: array
    Returns:
        a: array - activation prime
    '''
        
    return 1.0 - np.tanh(z)**2

### Loss Function

In [ ]:
# Helper function to evaluate the total loss on the dataset

def calculate_loss(model, X_l, y_l):
    '''
    Args:
        model: dictionay object containing weights and biases
        X_l: Feature Matrix
        y_l: Labels matrix (One-hot encoding expected)
    Returns:
        Average loss
    '''
    
    # Extract Weights and biases from the model
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    m = X_l.shape[0] # ----------------------------------------  plotting testing loss as well
    
    # Layer 1
    z1 = X_l.dot(W1) + b1  # Aggregation
    a1 = fn_actv(z1)     # Activation

    # Layer 2
    z2 = a1.dot(W2) + b2 # Aggregation
    a2 = fn_softmax(z2) # Activation
    
    # Calculating the loss
    data_loss = -(y_l * np.log(a2) + (1 - y_l) * np.log(1 - a2) ).sum() 

    return 1./m * data_loss

## Predict Function

For predictions, we will simply be using the forward propagation. No need to iterate or calculate the back propagation for supervised learning.


In [ ]:
# Helper function to predict an output

def predict(model, X_p):
    '''
     Args:
         model: dict object containing weights and biases
         X_p: Feature Matrix
    Returns:
        Predictions against the instances
         
    '''
    # Extract Weights and biases from the model
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']  
    
    #***********************************
    # Layer 1
    z1 = X_p.dot(W1) + b1  # Aggregation
    a1 = fn_actv(z1)     # Activation

    # Layer 2
    z2 = a1.dot(W2) + b2 # Aggregation
    a2 = fn_softmax(z2)     # Activation

   
    return np.argmax(a2, axis = 1)

**Note:** - following function will load model if exists, otherwise train from scratch

In [ ]:
# Prepare the Model

def build_model(nn_hdim, 
                X : np.ndarray, y : np.ndarray,
                X_t: np.ndarray, y_t: np.ndarray,
                alpha = 0.1,
                epochs = 20000, 
                print_loss=False):
    
    '''
    Args:
        nn_hdim : number of nodes in hidden layer
        X   : Training features to train
        y   : Trainig targets (labels)
        X_t : Testing features to train
        y_t : Testing targets (labels)
        alpha : learning rate
        epochs : Number of passes through the training data for gradient descent
        print_loss : If True, print the loss every nnn iterations
        
    Returns:
        Model: Dictionary object containing weights and biases
    '''
    if os.path.isfile(modelFilePath):
        print (f'Pre-trained model found at {modelFilePath}. Loading....')
        
        with open(modelFilePath, 'rb') as file:
            model_details = pickle.load(file)
            model = model_details['weights']
        print (f'Loaded existing model with\n {model_details["description"]}\n\n')
        print ('#'*50)
        W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
        
    else:
        print (f'No model found at {modelFilePath}. Initializing....')

        W1 = np.random.rand(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
        b1 = np.zeros((1, nn_hdim))
        
        W2 = np.random.rand(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
        b2 = np.zeros((1, nn_output_dim))
    
        model ={}
    
    startTime = time()
    
    curr_loss = 0
    epoch, loss, t_loss, acc, t_acc = [], [], [], [], []
    num_examples = X.shape[0] # training set size
    
    for i in range(0, epochs):
        
        epochTime = time()
        
        #### Forward ------------------------------
        # Layer 1
        z1 = X.dot(W1) + b1 
        a1 = fn_actv(z1)    # activation
        
        # Layer 2
        z2 = a1.dot(W2) + b2 
        a2 = fn_softmax(z2)    # activation
        
        
        ### Back Prop ------------------------------
        #Layer 2
        dz2 = a2- y
       
        dW2 = (a1.T).dot(dz2)  
        assert(W2.shape == dW2.shape), 'Shape of W2 {} and dW2 {} do not match'.format(W2.shape, dW2.shape)
        
        db2 = np.sum(dz2, axis=0, keepdims=True)
        assert(b2.shape == db2.shape), 'Shape of b2 {} and db2 {} do not match'.format(b2.shape, db2.shape)
        
        da1= dz2.dot(W2.T)
        
        #Layer 1
        dz1 = da1 * fn_actv_prime(a1)
        dW1 = (X.T).dot(dz1)
        assert(W1.shape == dW1.shape), 'Shape of W1 {} and dW1 {} do not match'.format(W1.shape, dW1.shape)
        
        db1 = np.sum(dz1, axis=0, keepdims=True)
        assert(b1.shape == db1.shape), 'Shape of b1 {} and db1 {} do not match'.format(b1.shape, db1.shape)
        
        
        # Weights update
        W1 += -alpha * dW1 /num_examples
        b1 += -alpha * db1 /num_examples
        
        W2 += -alpha * dW2 /num_examples
        b2 += -alpha * db2 /num_examples
        
        
        model = {'W1': W1, 'b1': b1,
                 'W2': W2, 'b2': b2}
        
        if i % 100 == 0:
            
            # Keep count of epoch
            epoch.append(i)
            
            # calculate loss
            curr_loss = calculate_loss (model, X, y)
            loss.append(curr_loss)
            
            ct_loss = calculate_loss ( model, X_t, y_t)
            t_loss.append(ct_loss)
            
            # calculate accuracy
            y_pred = predict(model, X)
            curr_acc = accuracy_score(np.argmax(y, axis = 1), y_pred )
            acc.append(curr_acc)
            
            yt_pred = predict(model, X_t)
            currt_acc = accuracy_score(np.argmax(y_t, axis = 1), yt_pred )
            t_acc.append(currt_acc)
            
        # Print the loss.
        if print_loss and i % 1000 == 0:
            print(f'Epoch # :{i}')
            print('Time of execution till now: {:0.2f} mins'.format( (time() - startTime ) / 60.) )
            print('Last epoch: {:0.2f} ms'.format ( (time() - epochTime) * 1000 ) )
            print('Train: Loss: %f  ---- Accuracy: %f' %(curr_loss, curr_acc))
            print('Test:  Loss: %f  ---- Accuracy: %f' %(ct_loss, currt_acc))
            print("-" *50)
    
    loss_hist['epoch'] = epoch
    loss_hist['loss'] = loss
    loss_hist['test_loss'] = t_loss
    loss_hist['acc'] = acc
    loss_hist['test_acc'] = t_acc
    
    return model

In [ ]:
nn_input_dim = X_train.shape[1] # input layer dimensionality
nn_output_dim = y_train.shape[1] # output layer dimensionality

nn_hidden_dim = 18

alpha = ALPHA

epochs = EPOCHS

In [ ]:
loss_hist = {}

modelFilePath = os.path.join(modelDir,subDir, 'S08_fifa_1L_swish_model.pkl')

model = build_model(nn_hidden_dim, X_train, y_train, X_test, y_test, alpha, epochs, print_loss = True) 

In [ ]:
loss_df = pd.DataFrame(loss_hist)
loss_df.shape

In [ ]:
model_details = {}

model_details['description'] = f'''Hidden layers : 1;
Layer sizes: 
    Input-{nn_input_dim}; Hidden-{nn_hidden_dim},Output-{nn_output_dim}
Activation Function :  Swish'''

model_details['weights'] = model

if SAVE_MODEL:
    
    with open(modelFilePath, 'wb') as file:
        pickle.dump(model_details, file, protocol=pickle.HIGHEST_PROTOCOL)
        
    loss_df.to_csv(os.path.join(modelDir, subDir, 'S07_fifa_1L_model_loss.csv'))

In [ ]:
print ('Model Weights:\n', model.keys())

In [ ]:
fig, axes = plt.subplots(1,2 , figsize = (15,6),)

l_range = 5 # ignoring first few records
x_var = 'epoch'
y1_var = 'loss'
y2_var = 'test_loss'

ax = axes[0]

loss_df[l_range:].plot(x = x_var, y = [y1_var, y2_var], ax = ax)

# get values for plotting
st_loss = loss_df[y1_var].iloc[0]
end_loss = loss_df[y1_var].iloc[-1]
t_st_loss = loss_df[y2_var].iloc[0]
t_end_loss = loss_df[y2_var].iloc[-1]

# little beautification
train_txtstr = "Train Loss: \n  Start : {:7.4f}\n    End : {:7.4f}".format( st_loss, end_loss )    # Train text to plot

test_txtstr = "Test Loss: \n  Start : {:7.4f}\n    End : {:7.4f}".format( t_st_loss, t_end_loss ) # Test text to plot
txtstr = ' {} \n{}'.format(train_txtstr, test_txtstr)
# properties  matplotlib.patch.Patch 
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords

ax.text(0.4, 0.95, txtstr, transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=props)

ax.set_xlabel(x_var)
ax.set_ylabel(y1_var)
ax.set_title('Overall')
ax.grid();

ax = axes[1]

y1_var = 'acc'
y2_var = 'test_acc'

loss_df[l_range:].plot(x = x_var, y = [y1_var, y2_var], ax = ax)

# get values for plotting
st_loss = loss_df[y1_var].iloc[0]
end_loss = loss_df[y1_var].iloc[-1]
t_st_loss = loss_df[y2_var].iloc[0]
t_end_loss = loss_df[y2_var].iloc[-1]

# little beautification
train_txtstr = "Train Accuracy: \n  Start : {:7.4f}\n    End : {:7.4f}".format( st_loss, end_loss )    # Train text to plot

test_txtstr = "Test Accuracy: \n  Start : {:7.4f}\n    End : {:7.4f}".format( t_st_loss, t_end_loss ) # Test text to plot# properties  matplotlib.patch.Patch 
txtstr = ' {} \n{}'.format(train_txtstr, test_txtstr)
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords

ax.text(0.4, 0.95, txtstr, transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=props)

ax.set_xlabel(x_var)
ax.set_ylabel(y1_var)
ax.set_title('Accuracies'.format(l_range))
ax.grid();
plt.tight_layout()

In [ ]:
y_pred = predict(model, X_train)
print('Accuracy score on Train Data :{:.5f}'.format( accuracy_score(np.argmax(y_train, axis = 1), y_pred ) ) )

In [ ]:
print(classification_report(np.argmax(y_train, axis = 1), y_pred))

In [ ]:
y_pred = predict(model, X_test)

print('Accuracy score on Test Data :{:.5f}'.format(accuracy_score(np.argmax(y_test, axis = 1), y_pred)))

In [ ]:
print(classification_report(np.argmax(y_test, axis = 1), y_pred))

In [ ]:
cm  = confusion_matrix(np.argmax(y_test, axis = 1), y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=labels.values())

fig, ax = plt.subplots(figsize = (4,4))

disp.plot(ax = ax, cmap = 'Blues', xticks_rotation = 'vertical', colorbar=False)

plt.show();

In [ ]:
loss_df[loss_df['test_loss'] == loss_df['test_loss'].min()]